In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

In [0]:
# Graph Clear
tf.reset_default_graph()

# tf Graph Input
X = tf.placeholder(tf.float32, [None, 784], name='InputData')
X_img = tf.reshape(X, shape=[-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

###################
# encoder

# conv layer
W1 = tf.Variable(tf.random_normal([5, 5, 1, 25], stddev=0.01))
b1 = tf.Variable(tf.random_normal([25], stddev=0.01))
h1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
h1 = tf.nn.bias_add(h1, b1)
h1 = tf.nn.relu(h1)

# pooling layer + dropout
p1 = tf.nn.max_pool(h1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
do1 = tf.nn.dropout(p1, keep_prob=0.75)

# fully connected layer
flat2 = tf.reshape(do1, shape=[-1, 14 * 14 * 25])

# fc1
W2 = tf.Variable(tf.random_normal([14 * 14 * 25, 14 * 14 * 5], stddev=0.01))
b2 = tf.Variable(tf.random_normal([14 * 14 * 5], stddev=0.01))
fc2 = tf.nn.relu(tf.add(tf.matmul(flat2, W2), b2))
do2 = tf.nn.dropout(fc2, keep_prob=0.75)

# fc2
W3 = tf.Variable(tf.random_normal([14 * 14 * 5, 14 * 14], stddev=0.01))
b3 = tf.Variable(tf.random_normal([14 * 14], stddev=0.01))
fc3 = tf.nn.relu(tf.add(tf.matmul(do2, W3), b3))

###################
# decoder

# fc2
W4 = tf.Variable(tf.random_normal([14 * 14, 14 * 14 * 5], stddev=0.01))
b4 = tf.Variable(tf.random_normal([14 * 14 * 5], stddev=0.01))
fc4 = tf.nn.relu(tf.add(tf.matmul(fc3, W4), b4))
do4 = tf.nn.dropout(fc4, keep_prob=0.75)

# fc1
W5 = tf.Variable(tf.random_normal([14 * 14 * 5, 14 * 14 * 25], stddev=0.01))
b5 = tf.Variable(tf.random_normal([14 * 14 * 25], stddev=0.01))
fc5 = tf.nn.relu(tf.add(tf.matmul(do4, W5), b5))
do5 = tf.nn.dropout(fc5, keep_prob=0.75)
do5 = tf.reshape(do5, shape=[-1, 14, 14, 25])

# deconv layer
dc1 = tf.contrib.layers.conv2d_transpose(do5, 
                                         num_outputs=25, 
                                         kernel_size=[5, 5],
                                         stride=[1, 1],
                                         padding='SAME',
                                         weights_initializer=tf.contrib.layers.xavier_initializer_conv2d(uniform=False),
                                         biases_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                                         activation_fn=tf.nn.relu)

# up scaling
size = [int(dc1.shape[1] * 2), int(dc1.shape[2] * 2)]
up1 = tf.image.resize_bilinear(dc1, size=size, align_corners=None)
up1 = tf.reshape(up1, [-1, 28 * 28 * 25])

W6 = tf.Variable(tf.random_normal([28 * 28 * 25, 28 * 28], stddev=0.01), dtype=tf.float32)
b6 = tf.Variable(tf.random_normal([28 * 28], stddev=0.01))
output = tf.nn.relu(tf.add(tf.matmul(up1, W6), b6))

with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.pow(output - X, 2))

with tf.name_scope('opt'):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)


In [0]:
n_epochs = 1
batch_size = 500

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(n_epochs):
    n_batches = int(mnist.train.num_examples / batch_size)

    # Loop over all batches
    for i in range(n_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={X: batch_x})

        if (i + 1) % 2 == 0:
            c = sess.run(cost, feed_dict={X: batch_x})
            print('Records: %d, cost: %.4f' % ((i + 1) * batch_size, c))

    # Display logs per epoch step
    print('Epoch: %d / %d,  cost: %.4f' % (epoch + 1, n_epochs, c))
print('Optimization Finished')

In [0]:
# Testing
# Encode and decode images from test set and visualize their reconstruction.
n = 4
canvas_orig = np.empty((28 * n, 28 * n))
canvas_recon = np.empty((28 * n, 28 * n))
for i in range(n):
    # MNIST test set
    batch_x, _ = mnist.test.next_batch(n)
    # Encode and decode the digit image
    g = sess.run(output, feed_dict={X: batch_x})
    
    # Display original images
    for j in range(n):
        # Draw the generated digits
        canvas_orig[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = batch_x[j].reshape([28, 28])
    # Display reconstructed images
    for j in range(n):
        # Draw the generated digits
        canvas_recon[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = g[j].reshape([28, 28])

print("Original Images")     
plt.figure(figsize=(n, n))
plt.imshow(canvas_orig, origin="upper", cmap="gray")
plt.show()

print("Reconstructed Images")
plt.figure(figsize=(n, n))
plt.imshow(canvas_recon, origin="upper", cmap="gray")
plt.show()

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np

def plot_conv_weights(weights, input_channel=0):
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        w = sess.run(weights)

        w_min = np.min(w)
        w_max = np.max(w)

        num_filters = w.shape[3]
        num_grids = math.ceil(math.sqrt(num_filters))
        fig, axes = plt.subplots(num_grids, num_grids)

        for i, ax in enumerate(axes.flat):
            if i < num_filters:
                img = w[:, :, input_channel, i]
                ax.imshow(img, vmin=w_min, vmax=w_max, interpolation='nearest', cmap='seismic')

            ax.set_xticks([])
            ax.set_yticks([])

        plt.show()
        
def plot_conv_layer(layer, image):

    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())

        feed_dict = {X: [image]}
        values = sess.run(layer, feed_dict=feed_dict)

        num_filters = values.shape[3]
        num_grids = math.ceil(math.sqrt(num_filters))
        fig, axes = plt.subplots(num_grids, num_grids)

        for i, ax in enumerate(axes.flat):
            if i < num_filters:
                img = values[0, :, :, i]
                ax.imshow(img, interpolation='nearest', cmap='binary')
                
            ax.set_xticks([])
            ax.set_yticks([])

        plt.show()